In [5]:
import bqplot
import bqplot.pyplot as blt
import numpy as np
import ipywidgets as ipy
import pandas as pd

In [6]:
fig = blt.figure()
fig.layout.width = "800px"
fig.layout.height = "400px"
line = blt.plot([1, 2, 3], [4, 5, 6])
scatter = blt.scatter([1, 2, 3], [4, 5, 6])
blt.lasso_selector()
blt.show()

Seems like need to install `bqplot` in the Jupyterlab extensions tab for this to work out.

Edit: Worked!

Note: The examples included in the github repo are much more complete than the docs themselves. Using them instead.

## One small step: Toggle the colour of selected points

In [7]:
df = pd.DataFrame({
    "x": np.linspace(0, 10, 50),
    "y": np.sin(np.linspace(0, 10, 50))
})

In [24]:
scales = {"x": bqplot.LinearScale(), "y": bqplot.LinearScale()}

In [38]:
scatter = bqplot.Scatter(
    x=df["x"],
    y=df["y"],
    scales=scales,
    # colors=["orange"],
    selected_style={
        "opacity": "1",
        "color": "orange",
    },
    unselected_style={
        "opacity": "0.2",
        "color": "gray"
    },
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter]
)

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter],
    axes=[x_ax, x_ay],
    title="""Brush Selector Example. Click and drag on the Figure to action.""",
    interaction=sel,
)

# Add HTML boxes to print stuff out
sel_html = ipy.HTML()
scatter_html = ipy.HTML()

# 2 steps for interactivity!
# Create linked function, then link it
def update_sel_text(*args):
    # *args is needed because the callback fxn is passed
    # lots of context metadata
    # so this is actually a (2,2) numpy array or None!
    sel_html.value = str(sel.selected)
    
def update_scatter_text(*args):
    # scatter.selected is a 1D numpy array or None
    scatter_html.value = str(scatter.selected)

# Run fxn when this attribute changes?
sel.observe(update_sel_text, "selected")
scatter.observe(update_scatter_text, "selected")
    
ipywidgets.VBox([fig, sel_html, scatter_html])

Hmm, the `selected` attribute resets when I escape the selector. Nothing stops me from writing some custom callback logic here, but I also know that there's a `MultiSelector` callback.

In [36]:
scatter = bqplot.Scatter(
    x=df["x"],
    y=df["y"],
    scales=scales,
    # colors=["orange"],
    selected_style={
        "opacity": "1",
        "color": "orange",
    },
    unselected_style={
        "opacity": "0.2",
        "color": "gray"
    },
)

sel = bqplot.interacts.MultiSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter]
)

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter],
    axes=[x_ax, x_ay],
    title="""Brush Selector Example. Click and drag on the Figure to action.""",
    interaction=sel,
)

# Add HTML boxes to print stuff out
sel_html = ipy.HTML()
scatter_html = ipy.HTML()

# 2 steps for interactivity!
# Create linked function, then link it
def update_sel_text(*args):
    # *args is needed because the callback fxn is passed
    # lots of context metadata
    # so this is actually a (2,2) numpy array or None!
    sel_html.value = str(sel.selected)
    
def update_scatter_text(*args):
    # scatter.selected is a 1D numpy array or None
    scatter_html.value = str(scatter.selected)

# Run fxn when this attribute changes?
sel.observe(update_sel_text, "selected")
scatter.observe(update_scatter_text, "selected")
    
ipywidgets.VBox([fig, sel_html, scatter_html])

Yeah nah. This is a 1D selector. Stick to writing custom logic.

### Writing the toggle selector

I don't think I can mess with the default `selected` attribute. But I can still plot two scatters.

In [40]:
df["selected"] = False

In [53]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["blue"],
    opacity=[1]
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1]
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
    brushing=True
)

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=sel,
)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            for i in idxs_to_toggle:
                existing_idx = df.index[i]
                df.loc[existing_idx, "selected"] = ~df.loc[existing_idx, "selected"]

        # Refresh xy's
        scatter.x = df.loc[df["selected"] == False, "x"]
        scatter.y = df.loc[df["selected"] == False, "y"]
        selected_scatter.x = df.loc[df["selected"] == True, "x"]
        selected_scatter.y = df.loc[df["selected"] == True, "y"]

# Trying to figure out how brushing works
html_div = ipy.HTML()        
        
def is_brushing(*args):
    html_div.value = f"sel.brushing: {sel.brushing}"
    
# This always returns True!
# scatter.observe(is_brushing, "selected")

# scatter.observe(update_toggle_selector, "selected")
sel.observe(update_toggle_selector, "brushing")
    
ipywidgets.VBox([fig, html_div])

Embarassingly, the points that end up changing colour aren't the same as the points that are being selected.

...

Ah. The indices of the two separate scatters are not the same as the global indices.

In [64]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["blue"],
    opacity=[1]
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1]
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
    brushing=True
)

panzoom = bqplot.interacts.PanZoom()

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=panzoom,
)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == False].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = True
                
        # Update the manual "selected" toggle
        idxs_to_toggle = selected_scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == True].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = False

        # Refresh xy's
        # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
        with scatter.hold_sync():
            scatter.x = df.loc[df["selected"] == False, "x"]
            scatter.y = df.loc[df["selected"] == False, "y"]
            
        with selected_scatter.hold_sync():
            selected_scatter.x = df.loc[df["selected"] == True, "x"]
            selected_scatter.y = df.loc[df["selected"] == True, "y"]

# Trying to figure out how brushing works
html_div = ipy.HTML()        
        
def currently_selected(*args):
    html_div.value = f"""
    Currently scatter.selected: {scatter.selected} <br>
    Currently selected_scatter.selected: {selected_scatter.selected} <br>
    df["selected"].value_counts(): {df["selected"].value_counts()}
    """
    
scatter.observe(currently_selected, "selected")

# scatter.observe(update_toggle_selector, "selected")
sel.observe(update_toggle_selector, "brushing")
    
ipywidgets.VBox([fig, html_div])

Now this works, but the plot flickers when redrawing something as simple as this. Hmm.

Using `hold_sync` fixed the issue. Hah!

In [57]:
df["selected"].value_counts()

selected
True     33
False    17
Name: count, dtype: int64

Added showing value_counts() in live refresh. The dataframe is indeed being changed on the fly. The core concept works! Hahahaha.

Now I'm thinking about scroll / zoom.

In [82]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["blue"],
    opacity=[1]
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1]
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
)

# Expects scales in a slightly different format
# ned to wrap in lists
panzoom = bqplot.interacts.PanZoom(scales={
    "x": [scales["x"]],
    # "y": [scales["y"]]
})

dropdown = ipywidgets.Dropdown(
    description="Tool", options=["pan/zoom", "box select"],
    value="box select"
)

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=panzoom,
)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == False].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = True
                
        # Update the manual "selected" toggle
        idxs_to_toggle = selected_scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == True].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = False

        # Refresh xy's
        # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
        with scatter.hold_sync():
            scatter.x = df.loc[df["selected"] == False, "x"]
            scatter.y = df.loc[df["selected"] == False, "y"]
            
        with selected_scatter.hold_sync():
            selected_scatter.x = df.loc[df["selected"] == True, "x"]
            selected_scatter.y = df.loc[df["selected"] == True, "y"]

# Trying to figure out how brushing works
html_div = ipy.HTML()        
        
def currently_selected(*args):
    html_div.value = f"""
    Currently scatter.selected: {scatter.selected} <br>
    Currently selected_scatter.selected: {selected_scatter.selected} <br>
    df["selected"].value_counts(): {df["selected"].value_counts()}
    """
    
scatter.observe(currently_selected, "selected")
dropdown.observe(update_tool, "value")
sel.observe(update_toggle_selector, "brushing")

ipywidgets.VBox([fig, html_div])

I found out that only a single interaction thingy can be passed ot the plot at any given time. This would be a limitation if I can't swap it out on the fly. I think it is possible, but gotta give it a try.

In [85]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["blue"],
    opacity=[1]
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1]
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
)

panzoom = bqplot.interacts.PanZoom(
    scales={"x": [scales["x"]]}
)

dropdown = ipywidgets.Dropdown(
    description="Tool", options=["pan/zoom", "box select"],
    value="box select"
)

interactions = {
    "pan/zoom": panzoom,
    "box select": sel
}

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=interactions.get(dropdown.value),
)

toolbar = bqplot.Toolbar(figure=fig)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == False].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = True
                
        # Update the manual "selected" toggle
        idxs_to_toggle = selected_scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == True].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = False

        # Refresh xy's
        # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
        with scatter.hold_sync():
            scatter.x = df.loc[df["selected"] == False, "x"]
            scatter.y = df.loc[df["selected"] == False, "y"]
            
        with selected_scatter.hold_sync():
            selected_scatter.x = df.loc[df["selected"] == True, "x"]
            selected_scatter.y = df.loc[df["selected"] == True, "y"]

def update_tool(*args):
    fig.interaction = interactions.get(dropdown.value)
            
# Trying to figure out how brushing works
html_div = ipy.HTML()        
        
def currently_selected(*args):
    html_div.value = f"""
    Currently scatter.selected: {scatter.selected} <br>
    Currently selected_scatter.selected: {selected_scatter.selected} <br>
    df["selected"].value_counts(): {df["selected"].value_counts()}
    """
    
scatter.observe(currently_selected, "selected")
dropdown.observe(update_tool, "value")
sel.observe(update_toggle_selector, "brushing")

ipywidgets.VBox([dropdown, fig, html_div])

Zoom can be set along x-axis only, y-axis only, or both. A bit clunky but it's all good. Problem I have now, is I don't know how to reset.

Found that I can directly mess with the Scales.

In [94]:
scales["y"].min = -1
scales["x"].min = +1

Okay.

The plot is a bit heavy to pan. Maybe because I've been recreating the plot again and again, and all of them are linked?